<a href="https://colab.research.google.com/github/datascisteven/greenflash_llm_project/blob/main/Greenflash_Toxicity_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Packages

In [ ]:
!pip install -q sentencepiece torch datasets transformers huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# prompt: pandas option to show all columns

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
# prompt: Authenticate with huggingface

from huggingface_hub import notebook_login
notebook_login()


# Jigsaw Dataset

- **Toxic:** very bad, unpleasant, or harmful

- **Severe toxic:** extremely bad and offensive

- **Obscene:** (of the portrayal or description of sexual matters) offensive or disgusting by accepted standards of morality and decency

- **Threat:** a statement of an intention to inflict pain, injury, damage, or other hostile action on someone in retribution for something done or not done

- **Insult:** speak to or treat with disrespect or scornful abuse

- **Identity hate:** hatred, hostility, or violence towards members of a race, ethnicity, nation, religion, gender, gender identity, sexual orientation or any other designated sector of society


In [ ]:
from google.colab import files

files.upload() #this will prompt you to upload the kaggle.json.



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"datascisteven","key":"ac08eb6fb04fd93b3e978833f15ba72c"}'}

In [ ]:
#make sure kaggle.json file is present
!ls -lha kaggle.json


# Install kaggle API client
!pip install -q kaggle

#kaggle API client expects the file to be in ~/.kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#we need to set permissions
!chmod 600 /root/.kaggle/kaggle.json

#check your directory before downloading the datasets
!pwd

!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

# If your file is a zip file you can unzip with the following code
!unzip jigsaw-toxic-comment-classification-challenge.zip

!unzip train.csv.zip
!unzip test.csv.zip
!unzip sample_submission.csv.zip
!unzip test_labels.csv.zip

!mkdir /content/gdrive/MyDrive/Projects/Greenflash/Jigsaw\ Toxic\ Comment\ Classification\ Challenge
!cp /content/train.csv /content/gdrive/MyDrive/Projects/Greenflash/Jigsaw\ Toxic\ Comment\ Classification\ Challenge/
!cp /content/test.csv /content/gdrive/MyDrive/Projects/Greenflash/Jigsaw\ Toxic\ Comment\ Classification\ Challenge/
!cp /content/test_labels.csv /content/gdrive/MyDrive/Projects/Greenflash/Jigsaw\ Toxic\ Comment\ Classification\ Challenge/
!cp /content/sample_submission.csv /content/gdrive/MyDrive/Projects/Greenflash/Jigsaw\ Toxic\ Comment\ Classification\ Challenge/


-rw-r--r-- 1 root root 69 Sep 13 12:49 kaggle.json
/content


In [ ]:
all_data = pd.read_csv('/content/gdrive/MyDrive/Projects/Greenflash/Toxicity/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv')
all_data.head()

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,male,female,transgender,other_gender,heterosexual,homosexual_gay_or_lesbian,bisexual,other_sexual_orientation,christian,jewish,muslim,hindu,buddhist,atheist,other_religion,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
0,1083994,He got his money... now he lies in wait till a...,train,2017-03-06 15:21:53.675241+00,21,NaN,317120,approved,0,0,0,2,0,0.373134,0.044776,0.089552,0.014925,0.000000,0.343284,0.014925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,67
1,650904,Mad dog will surely put the liberals in mental...,train,2016-12-02 16:44:21.329535+00,21,NaN,154086,approved,0,0,1,2,0,0.605263,0.013158,0.065789,0.013158,0.092105,0.565789,0.065789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
2,5902188,And Trump continues his lifelong cowardice by ...,train,2017-09-05 19:05:32.341360+00,55,NaN,374342,approved,1,0,2,3,7,0.666667,0.015873,0.031746,0.000000,0.047619,0.666667,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,63
3,7084460,"""while arresting a man for resisting arrest"".\...",test,2016-11-01 16:53:33.561631+00,13,NaN,149218,approved,0,0,0,0,0,0.815789,0.065789,0.552632,0.592105,0.000000,0.684211,0.105263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
4,5410943,Tucker and Paul are both total bad ass mofo's.,train,2017-06-14 05:08:21.997315+00,21,NaN,344096,approved,0,0,0,1,0,0.550000,0.037500,0.337500,0.275000,0.037500,0.487500,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,80


In [ ]:
train_data = all_data.loc[all_data.split=='train']
train_data.head()

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,male,female,transgender,other_gender,heterosexual,homosexual_gay_or_lesbian,bisexual,other_sexual_orientation,christian,jewish,muslim,hindu,buddhist,atheist,other_religion,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
0,1083994,He got his money... now he lies in wait till a...,train,2017-03-06 15:21:53.675241+00,21,NaN,317120,approved,0,0,0,2,0,0.373134,0.044776,0.089552,0.014925,0.000000,0.343284,0.014925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,67
1,650904,Mad dog will surely put the liberals in mental...,train,2016-12-02 16:44:21.329535+00,21,NaN,154086,approved,0,0,1,2,0,0.605263,0.013158,0.065789,0.013158,0.092105,0.565789,0.065789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
2,5902188,And Trump continues his lifelong cowardice by ...,train,2017-09-05 19:05:32.341360+00,55,NaN,374342,approved,1,0,2,3,7,0.666667,0.015873,0.031746,0.000000,0.047619,0.666667,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,63
4,5410943,Tucker and Paul are both total bad ass mofo's.,train,2017-06-14 05:08:21.997315+00,21,NaN,344096,approved,0,0,0,1,0,0.550000,0.037500,0.337500,0.275000,0.037500,0.487500,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,80
5,6290444,"Cry me a river, why don't you.\nDrinking, drug...",train,2017-11-04 22:04:11.596185+00,54,6290143.0,396946,rejected,0,0,0,0,0,0.203390,0.016949,0.050847,0.000000,0.033898,0.203390,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,59


In [ ]:
import numpy as np

train_data[1] = np.where(train_data.toxicity > 0.5, 1, 0)
train_data[2] = np.where(train_data.severe_toxicity > 0.5, 1, 0)
train_data[3] = np.where(train_data.obscene > 0.5, 1, 0)
train_data[4] = np.where(train_data.sexual_explicit > 0.5, 1, 0)
train_data[5] = np.where(train_data.identity_attack > 0.5, 1, 0)
train_data[6] = np.where(train_data.insult > 0.5, 1, 0)
train_data[7] = np.where(train_data.threat > 0.5, 1, 0)

train_data[8] = np.where(train_data.male > 0.5, 1, 0)
train_data[9] = np.where(train_data.female > 0.5, 1, 0)
train_data[10] = np.where(train_data.transgender > 0.5, 1, 0)
train_data[11] = np.where(train_data.other_gender > 0.5, 1, 0)

train_data[12] = np.where(train_data.heterosexual > 0.5, 1, 0)
train_data[13] = np.where(train_data.homosexual_gay_or_lesbian > 0.5, 1, 0)
train_data[14] = np.where(train_data.bisexual > 0.5, 1, 0)
train_data[15] = np.where(train_data.other_sexual_orientation > 0.5, 1, 0)

train_data[16] = np.where(train_data.christian > 0.5, 1, 0)
train_data[17] = np.where(train_data.jewish > 0.5, 1, 0)
train_data[18] = np.where(train_data.muslim > 0.5, 1, 0)
train_data[19] = np.where(train_data.hindu > 0.5, 1, 0)
train_data[20] = np.where(train_data.buddhist > 0.5, 1, 0)
train_data[21] = np.where(train_data.atheist > 0.5, 1, 0)
train_data[22] = np.where(train_data.other_religion > 0.5, 1, 0)

train_data[23] = np.where(train_data.black > 0.5, 1, 0)
train_data[24] = np.where(train_data.white > 0.5, 1, 0)
train_data[25] = np.where(train_data.asian > 0.5, 1, 0)
train_data[26] = np.where(train_data.latino > 0.5, 1, 0)
train_data[27] = np.where(train_data.other_race_or_ethnicity > 0.5, 1, 0)

train_data[28] = np.where(train_data.physical_disability > 0.5, 1, 0)
train_data[29] = np.where(train_data.intellectual_or_learning_disability > 0.5, 1, 0)
train_data[30] = np.where(train_data.psychiatric_or_mental_illness > 0.5, 1, 0)
train_data[31] = np.where(train_data.other_disability > 0.5, 1, 0)


columns = ['toxicity',
           'severe_toxicity',
           'obscene',
           'sexual_explicit',
           'identity_attack',
           'insult',
           'threat',
           'male',
            'female',
            'transgender',
            'other_gender',
            'heterosexual',
            'homosexual_gay_or_lesbian',
            'bisexual',
            'other_sexual_orientation',
            'christian',
            'jewish',
            'muslim',
            'hindu',
            'buddhist',
            'atheist',
            'other_religion',
            'black',
            'white',
            'asian',
            'latino',
            'other_race_or_ethnicity',
            'physical_disability',
            'intellectual_or_learning_disability',
            'psychiatric_or_mental_illness',
            'other_disability']

train_data = train_data.drop(columns=columns)

train_js = pd.concat([train_data[['id', 'comment_text']], train_data.iloc[:, -31:]], axis=1)
train_js.columns = ['id', 'comment_text'] + columns
train_js.head()

<ipython-input-8-037b4bdb389d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[1] = np.where(train_data.toxicity > 0.5, 1, 0)
<ipython-input-8-037b4bdb389d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[2] = np.where(train_data.severe_toxicity > 0.5, 1, 0)
<ipython-input-8-037b4bdb389d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

,id,comment_text,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,male,female,transgender,other_gender,heterosexual,homosexual_gay_or_lesbian,bisexual,other_sexual_orientation,christian,jewish,muslim,hindu,buddhist,atheist,other_religion,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability
0,1083994,He got his money... now he lies in wait till a...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,650904,Mad dog will surely put the liberals in mental...,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5902188,And Trump continues his lifelong cowardice by ...,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5410943,Tucker and Paul are both total bad ass mofo's.,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,6290444,"Cry me a river, why don't you.\nDrinking, drug...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_js['race'] = train_js[['asian', 'black', 'latino', 'other_race_or_ethnicity', 'white']].sum(axis=1)
train_js['gender'] = train_js[['female', 'male']].sum(axis=1)
train_js['sexual_orientation'] = train_js[['bisexual', 'heterosexual', 'homosexual_gay_or_lesbian', 'transgender', 'other_sexual_orientation', 'other_gender']].sum(axis=1)
train_js['religion'] = train_js[['atheist', 'buddhist', 'christian', 'hindu', 'jewish', 'muslim', 'other_religion']].sum(axis=1)
train_js['ability'] = train_js[['physical_disability', 'intellectual_or_learning_disability', 'psychiatric_or_mental_illness', 'other_disability']].sum(axis=1)

tr_js = pd.concat([train_js.iloc[:, :9], train_js.iloc[:, -5:]], axis=1)
tr_js.head()

,id,comment_text,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,race,gender,sexual_orientation,religion,ability
0,1083994,He got his money... now he lies in wait till a...,0,0,0,0,0,0,0,0,0,0,0,0
1,650904,Mad dog will surely put the liberals in mental...,1,0,0,0,0,1,0,0,0,0,0,0
2,5902188,And Trump continues his lifelong cowardice by ...,1,0,0,0,0,1,0,0,0,0,0,0
4,5410943,Tucker and Paul are both total bad ass mofo's.,1,0,0,0,0,0,0,0,0,0,0,0
5,6290444,"Cry me a river, why don't you.\nDrinking, drug...",0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
tr_js['race'] = tr_js.race.replace(2, 1).replace(3, 1).replace(4, 1).replace(5, 1).replace(6, 1)
tr_js['gender'] = tr_js.gender.replace(2, 1). replace(3, 1)
tr_js['sexual_orientation'] = tr_js.sexual_orientation.replace(2, 1).replace(3, 1).replace(4, 1)
tr_js['religion'] = tr_js.religion.replace(2, 1).replace(3, 1).replace(4, 1).replace(5, 1).replace(6, 1)
tr_js['ability'] = tr_js.ability.replace(2, 1).replace(3, 1)
tr_js.head()

,id,comment_text,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,race,gender,sexual_orientation,religion,ability
0,1083994,He got his money... now he lies in wait till a...,0,0,0,0,0,0,0,0,0,0,0,0
1,650904,Mad dog will surely put the liberals in mental...,1,0,0,0,0,1,0,0,0,0,0,0
2,5902188,And Trump continues his lifelong cowardice by ...,1,0,0,0,0,1,0,0,0,0,0,0
4,5410943,Tucker and Paul are both total bad ass mofo's.,1,0,0,0,0,0,0,0,0,0,0,0
5,6290444,"Cry me a river, why don't you.\nDrinking, drug...",0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
def find_float_in_column(df, column_name):
    """Finds rows with float values in a column of strings."""

    def is_float(x):
        try:
            float(x)
            return True
        except ValueError:
            return False

    return df[df[column_name].apply(is_float)]

# Example usage
result_df = find_float_in_column(tr_js, 'comment_text')
print(result_df)

              id comment_text  toxicity  severe_toxicity  obscene  \
37007     453169            5         0                0        0   
43926     600543        11111         0                0        0   
86992    5772784           +1         0                0        0   
97604    6056691         1492         0                0        0   
99027    6321733           42         0                0        0   
178602   5061970            1         0                0        0   
185077   5122871            0         0                0        0   
192837   6216305         2018         0                0        0   
234361   6287227           +1         0                0        0   
247958   5225146            7         0                0        0   
251265   5143584            0         0                0        0   
412980   6322637          007         0                0        0   
446630    392337          NaN         0                0        0   
456011    716706         1981     

In [ ]:
tr_js = tr_js.drop(result_df.index)
tr_js.head()

,id,comment_text,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,race,gender,sexual_orientation,religion,ability
0,1083994,He got his money... now he lies in wait till a...,0,0,0,0,0,0,0,0,0,0,0,0
1,650904,Mad dog will surely put the liberals in mental...,1,0,0,0,0,1,0,0,0,0,0,0
2,5902188,And Trump continues his lifelong cowardice by ...,1,0,0,0,0,1,0,0,0,0,0,0
4,5410943,Tucker and Paul are both total bad ass mofo's.,1,0,0,0,0,0,0,0,0,0,0,0
5,6290444,"Cry me a river, why don't you.\nDrinking, drug...",0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
def find_float_in_column(df, column_name):
    """Finds rows with float values in a column of strings."""

    def is_float(x):
        try:
            float(x)
            return True
        except ValueError:
            return False

    return df[df[column_name].apply(is_float)]

# Example usage
result_df = find_float_in_column(tr_js, 'comment_text')
print(result_df)


Empty DataFrame
Columns: [id, comment_text, toxicity, severe_toxicity, obscene, sexual_explicit, identity_attack, insult, threat, race, gender, sexual_orientation, religion, ability]
Index: []


In [ ]:
tr_js['comment_text'] = tr_js['comment_text'].astype(str)

In [ ]:
tr_js['comment_text'].dtypes

dtype('O')

In [ ]:
tr_js['comment_text'] = tr_js['comment_text'].apply(lambda x: x if isinstance(x, str) else None)

In [ ]:
tr_js.comment_text.value_counts(dropna=False)

KeyboardInterrupt: 

In [ ]:
tr_js = tr_js.dropna(subset=['comment_text'])
tr_js.head()

,id,comment_text,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,race,gender,sexual_orientation,religion,ability
0,1083994,He got his money... now he lies in wait till a...,0,0,0,0,0,0,0,0,0,0,0,0
1,650904,Mad dog will surely put the liberals in mental...,1,0,0,0,0,1,0,0,0,0,0,0
2,5902188,And Trump continues his lifelong cowardice by ...,1,0,0,0,0,1,0,0,0,0,0,0
4,5410943,Tucker and Paul are both total bad ass mofo's.,1,0,0,0,0,0,0,0,0,0,0,0
5,6290444,"Cry me a river, why don't you.\nDrinking, drug...",0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
len(tr_js)

1804812

In [ ]:
tr_js['comment_text'].dtypes

dtype('O')

In [ ]:
tr_js.to_csv('jigsaw_train_revised.csv')
!cp jigsaw_train_revised.csv /content/gdrive/MyDrive/Projects/Greenflash/Toxicity/

In [ ]:
np.dtype('O')

In [ ]:
from datasets import load_dataset

ds = load_dataset("allenai/real-toxicity-prompts")

In [ ]:
toxicity_df = ds['train'].to_pandas()
toxicity_df.head()

In [ ]:
toxicity_df.prompt[0]

In [ ]:
toxicity_df.continuation[0]

## Ethos Dataset

In [ ]:
ethos = pd.read_csv('/content/gdrive/MyDrive/Projects/Greenflash/Toxicity/Ethos-Hate-Speech-Dataset-master/ethos/ethos_data/Ethos_Dataset_Multi_Label.csv', sep=";")
ethos.head()

,comment,violence,directed_vs_generalized,gender,race,national_origin,disability,religion,sexual_orientation
0,You should know women's sports are a joke,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0
1,You look like Sloth with deeper Down’s syndrome,0.000000,0.833333,0.000000,0.000000,0.000000,1.0,0.0,0.0
2,You look like Russian and speak like Indian. B...,1.000000,0.500000,0.142857,0.142857,0.714286,0.0,0.0,0.0
3,"Women deserve to be abused, I guess.",0.666667,0.166667,1.000000,0.000000,0.000000,0.0,0.0,0.0
4,Women are made for making babies and cooking d...,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0


In [ ]:
ethos[1] = np.where(ethos.gender > 0.5, 1, 0)
ethos[2] = np.where((ethos.race > 0.5) | (ethos.national_origin > 0.5), 1, 0)
ethos[3] = np.where(ethos.disability > 0.5, 1, 0)
ethos[4] = np.where(ethos.religion > 0.5, 1, 0)
ethos[5] = np.where(ethos.sexual_orientation > 0.5, 1, 0)
ethos[6] = np.where(ethos.violence > 0.5, 1, 0)

ethos.head()

,comment,violence,directed_vs_generalized,gender,race,national_origin,disability,religion,sexual_orientation,1,2,3,4,5,6
0,You should know women's sports are a joke,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0,1,0,0,0,0,0
1,You look like Sloth with deeper Down’s syndrome,0.000000,0.833333,0.000000,0.000000,0.000000,1.0,0.0,0.0,0,0,1,0,0,0
2,You look like Russian and speak like Indian. B...,1.000000,0.500000,0.142857,0.142857,0.714286,0.0,0.0,0.0,0,1,0,0,0,1
3,"Women deserve to be abused, I guess.",0.666667,0.166667,1.000000,0.000000,0.000000,0.0,0.0,0.0,1,0,0,0,0,1
4,Women are made for making babies and cooking d...,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0,1,0,0,0,0,0


In [ ]:
columns = ['gender', 'race', 'national_origin', 'disability', 'religion', 'sexual_orientation', 'directed_vs_generalized', 'violence']
ethos = ethos.drop(columns=columns)
ethos.columns = ['text', 'gender', 'race', 'ability', 'religion', 'sexual_orientation', 'violence']
ethos['toxicity'] = 1
ethos.head()

,text,gender,race,ability,religion,sexual_orientation,violence,toxicity
0,You should know women's sports are a joke,1,0,0,0,0,0,1
1,You look like Sloth with deeper Down’s syndrome,0,0,1,0,0,0,1
2,You look like Russian and speak like Indian. B...,0,1,0,0,0,1,1
3,"Women deserve to be abused, I guess.",1,0,0,0,0,1,1
4,Women are made for making babies and cooking d...,1,0,0,0,0,0,1


In [ ]:
len(ethos)

433

## GAB Hate Speech Corpus

In [ ]:
annotations = pd.read_csv('/content/gdrive/MyDrive/Projects/Greenflash/Toxicity/osfstorage-archive/Data/GabHateCorpus_annotations.tsv', sep='\t')

annotations.columns = ['id', 'annotator_id', 'text', 'hate', 'human_dignity', 'call_for_violence', 'vulgarity_offensive', 'religion', 'race_ethnicity', 'sexual_orientation', 'gender', 'ideology', 'nationality', 'political', 'disability', 'explicit', 'implicit']

annotations = annotations.loc[annotations.gender.notna()]

anno = annotations.groupby(['id', 'text'], as_index=False)[['hate', 'human_dignity', 'call_for_violence', 'vulgarity_offensive', 'religion', 'race_ethnicity', 'sexual_orientation', 'gender', 'ideology', 'nationality', 'political', 'disability']].sum()

anno.head()

,id,text,hate,human_dignity,call_for_violence,vulgarity_offensive,religion,race_ethnicity,sexual_orientation,gender,ideology,nationality,political,disability
0,3,most disgusting people: - third worlders - gay...,2,2,0,0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0
1,8,"New band names, by me: - Gay Retards on Fire -...",2,2,0,1,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,9,'Is fucking a retarded girl rape? Asking for a...,3,3,0,2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,12,The mayor of Philadelphia has blood on his ha...,1,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14,"Washington and Jefferson didn't import slaves,...",3,3,0,0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
anno['race'] = anno['race_ethnicity'] + anno['nationality']
anno = anno.drop(columns=['hate', 'human_dignity', 'political', 'ideology', 'race_ethnicity', 'nationality'])
anno.iloc[:, 2:] = anno.iloc[:, 2:].replace(2, 1).replace(3, 1).replace(4, 1).replace(5, 1).replace(6, 1)
anno.head()

,id,text,call_for_violence,vulgarity_offensive,religion,sexual_orientation,gender,disability,race
0,3,most disgusting people: - third worlders - gay...,0,0,0.0,1.0,0.0,0.0,1.0
1,8,"New band names, by me: - Gay Retards on Fire -...",0,1,1.0,1.0,0.0,0.0,0.0
2,9,'Is fucking a retarded girl rape? Asking for a...,0,1,0.0,0.0,0.0,1.0,1.0
3,12,The mayor of Philadelphia has blood on his ha...,0,0,0.0,0.0,0.0,0.0,0.0
4,14,"Washington and Jefferson didn't import slaves,...",0,0,1.0,0.0,0.0,0.0,1.0


In [ ]:
anno = anno.drop(anno.iloc[:, 2:].sum(axis=1).loc[anno.iloc[:, 2:].sum(axis=1)==0].index)
anno.head()

,id,text,call_for_violence,vulgarity_offensive,religion,sexual_orientation,gender,disability,race
0,3,most disgusting people: - third worlders - gay...,0,0,0.0,1.0,0.0,0.0,1.0
1,8,"New band names, by me: - Gay Retards on Fire -...",0,1,1.0,1.0,0.0,0.0,0.0
2,9,'Is fucking a retarded girl rape? Asking for a...,0,1,0.0,0.0,0.0,1.0,1.0
4,14,"Washington and Jefferson didn't import slaves,...",0,0,1.0,0.0,0.0,0.0,1.0
5,16,Whites are the ONLY people in this world that ...,0,1,0.0,0.0,0.0,0.0,1.0


In [ ]:
anno['toxicity'] = 1

In [ ]:
len(anno)

5672

# Perspective API

In [ ]:
import requests
import json

# Function to analyze text using Perspective API
def analyze_toxicity(api_key, text):
    url = f"https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze?key={api_key}"

    # Define the request payload
    data = {
        "comment": {"text": text},
        "languages": ["en"],
        "requestedAttributes": {
            "TOXICITY": {},
            "SEVERE_TOXICITY": {},
            "INSULT": {},
            "PROFANITY": {},
            "THREAT": {}
        }
    }

    # Make the POST request to the Perspective API
    response = requests.post(url, data=json.dumps(data), headers={'Content-Type': 'application/json'})

    # Parse the JSON response
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
api_key = 'YOUR_PERSPECTIVE_API_KEY'
response_text = "You are such an idiot."

# Analyze the response text for toxicity
result = analyze_toxicity(api_key, response_text)

# Output the result
print(json.dumps(result, indent=2))

In [ ]:
{
  "attributeScores": {
    "TOXICITY": {
      "spanScores": [
        {
          "begin": 0,
          "end": 20,
          "score": {
            "value": 0.87,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.87,
        "type": "PROBABILITY"
      }
    },
    "SEVERE_TOXICITY": {
      "spanScores": [
        {
          "begin": 0,
          "end": 20,
          "score": {
            "value": 0.67,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.67,
        "type": "PROBABILITY"
      }
    },
    "INSULT": {
      "spanScores": [
        {
          "begin": 0,
          "end": 20,
          "score": {
            "value": 0.82,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.82,
        "type": "PROBABILITY"
      }
    },
    "PROFANITY": {
      "spanScores": [
        {
          "begin": 0,
          "end": 20,
          "score": {
            "value": 0.58,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.58,
        "type": "PROBABILITY"
      }
    },
    "THREAT": {
      "spanScores": [
        {
          "begin": 0,
          "end": 20,
          "score": {
            "value": 0.15,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.15,
        "type": "PROBABILITY"
      }
    }
  },
  "languages": ["en"]
}

In [ ]:
"requestedAttributes": {
    "TOXICITY": {},
    "SEVERE_TOXICITY": {},
    "PROFANITY": {}
}

In [ ]:
def check_chatbot_response_for_toxicity(api_key, chatbot_response):
    result = analyze_toxicity(api_key, chatbot_response)

    toxicity_score = result['attributeScores']['TOXICITY']['summaryScore']['value']

    # Set a threshold, e.g., 0.7, to classify text as toxic
    if toxicity_score > 0.7:
        print(f"Chatbot response flagged as toxic: {chatbot_response}")
        return False
    else:
        print(f"Chatbot response is non-toxic: {chatbot_response}")
        return True

# Example chatbot response
chatbot_response = "You are the worst!"
check_chatbot_response_for_toxicity(api_key, chatbot_response)

In [ ]:
toxicity_threshold = 0.7
toxicity_score = result['attributeScores']['TOXICITY']['summaryScore']['value']

if toxicity_score > toxicity_threshold:
    print("Toxic response detected.")
else:
    print("Response is safe.")

NameError: name 'result' is not defined

# ToxiGen Dataset

In [ ]:
dataset = load_dataset("toxigen/toxigen-data", "annotated")

prompts = load_dataset("toxigen/toxigen-data", "prompts")

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/79.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/689k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/940 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/8960 [00:00<?, ? examples/s]

...

Generating hate_lgbtq_1k split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating hate_women_1k split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating neutral_middle_east_1k split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating neutral_native_american_1k split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating neutral_physical_disability_1k split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text', 'target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method'],
        num_rows: 940
    })
    train: Dataset({
        features: ['text', 'target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method'],
        num_rows: 8960
    })
})

In [ ]:
train = dataset['train']
train

Dataset({
    features: ['text', 'target_group', 'factual?', 'ingroup_effect', 'lewd', 'framing', 'predicted_group', 'stereotyping', 'intent', 'toxicity_ai', 'toxicity_human', 'predicted_author', 'actual_method'],
    num_rows: 8960
})

In [ ]:
prompts

DatasetDict({
    hate_trans_1k: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    neutral_black_1k: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    hate_native_american_1k: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    neutral_immigrant_1k: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    hate_middle_east_1k: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    neutral_lgbtq_1k: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    neutral_women_1k: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    neutral_chinese_1k: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    hate_latino_1k: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    hate_bisexual_1k: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    hate_mexican_1k: Dataset({
        features: ['text'],
        num_rows

In [ ]:
{
  "text": "Immigrants are ruining this country!",
  "label": "toxic"  # or "non-toxic"
}